In [1]:
import numpy as np
from utils import ClassEncoder
from datasets import get_iris_dataset
X_full_iris, y_full_iris = get_iris_dataset()
print(X_full_iris.shape)
print(y_full_iris.shape)

(150, 4)
(150, 1)


In [2]:
def print_distribution(_y):
    encoder = ClassEncoder()
    encoded_y = encoder.fit_transform(_y) # convert class to number (encode)
    print('Distribución de clases:')
    distribution = np.bincount(encoded_y.flatten())/len(encoded_y)
    for class_name, value in zip(encoder.names, distribution):
        print(f'{class_name}: {value:.4f}')

print_distribution(y_full_iris)


Distribución de clases:
setosa: 0.3333
versicolor: 0.3333
virginica: 0.3333


Se observa que el dataset Iris se encuentra balanceado, es decir que no hay alguna preponderancia de alguna de las clases por sobre las demás.

### 1) QDA Entrenado con:  probabilidades a priori uniforme y  una clase con probabilidad 0.9, las demás 0.05 ( 3 combinaciones)

In [3]:
from utils import split_transpose, QDA, accuracy

def priori_test(dataset):
    X_full, y_full = dataset
    a_priori_A = [1/3, 1/3, 1/3] # modelo 0
    a_priori_B_1 = [0.9, 0.05, 0.05] # modelo 1
    a_priori_B_2 = [0.05, 0.9, 0.05] # modelo 2
    a_priori_B_3 = [0.05, 0.05, 0.9] # modelo 3
    
    a_priori_list = [a_priori_A, a_priori_B_1, a_priori_B_2, a_priori_B_3]
    # from utils import QDA
    # rng_seed = 6543
    train_x, train_y, test_x, test_y = split_transpose(X_full, y_full, 0.4, 6543)
    
    for i,_a_priori in enumerate(a_priori_list):
        model = QDA()
        model.fit(train_x, train_y, _a_priori)
        print('A prioris:')
        print(",".join([f' {class_name}:{p:.3f} ' for class_name, p in zip(model.encoder.names, _a_priori)]))
        train_acc = accuracy(train_y, model.predict(train_x))
        test_acc = accuracy(test_y, model.predict(test_x))
        print(f"[Model {i}] Train (apparent) error is {1-train_acc:.4f} while test error is {1-test_acc:.4f}")
        # print('\n')

priori_test(get_iris_dataset())

A prioris:
 setosa:0.333 , versicolor:0.333 , virginica:0.333 
[Model 0] Train (apparent) error is 0.0222 while test error is 0.0167
A prioris:
 setosa:0.900 , versicolor:0.050 , virginica:0.050 
[Model 1] Train (apparent) error is 0.0222 while test error is 0.0167
A prioris:
 setosa:0.050 , versicolor:0.900 , virginica:0.050 
[Model 2] Train (apparent) error is 0.0333 while test error is 0.0000
A prioris:
 setosa:0.050 , versicolor:0.050 , virginica:0.900 
[Model 3] Train (apparent) error is 0.0333 while test error is 0.0500


A partir de estos datos y dejando de lado cuál es la verdadera distribución, se podrían hacer las siguientes suposiciones:
- El modelo 0 y el modelo 1 parecerían cometer el mismo grado de error al hacer dichas suposiciones sobre los priors.
- El modelo 2 parecería sobreajustar (hay overfitting) a los datos de test.
- El modelo 3 tiene un mayor error tanto en el entrenamiento como en la prueba, lo que indica que no logra generalizar y que dichos priors tienen un efecto detrimental en la performance del modelo.

Se utilizará la versión de QDA provista por SKLearn (a pesar de las diferencias en su implementación) a modo de evaluar cualitativamente el efecto de los priors sobre éste dataset y confirmar las suposiciones ya mencionadas, utilizando Cross Validation:

In [4]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.model_selection import cross_val_score

def priori_test_cross_val(dataset):
    X_full, y_full = dataset
    a_priori_A = [1/3, 1/3, 1/3] # modelo 0
    a_priori_B_1 = [0.9, 0.05, 0.05] # modelo 1
    a_priori_B_2 = [0.05, 0.9, 0.05] # modelo 2
    a_priori_B_3 = [0.05, 0.05, 0.9] # modelo 3
        
    a_priori_list = [a_priori_A, a_priori_B_1, a_priori_B_2, a_priori_B_3]
    
    for i, _priors in enumerate(a_priori_list):
        qda = QuadraticDiscriminantAnalysis(priors=_priors) #utilizo QDA de sklearn para poder utilizar cross_val_score sin problemas
        cv_scores = cross_val_score(qda, X_full, y_full.flatten(), cv=5)  # 5-fold cross-validation
        print(f"Accuracy with priors {_priors}: {cv_scores.mean()} ± {cv_scores.std()} for model {i}")

priori_test_cross_val(get_iris_dataset())

Accuracy with priors [0.3333333333333333, 0.3333333333333333, 0.3333333333333333]: 0.9800000000000001 ± 0.02666666666666666 for model 0
Accuracy with priors [0.9, 0.05, 0.05]: 0.9800000000000001 ± 0.02666666666666666 for model 1
Accuracy with priors [0.05, 0.9, 0.05]: 0.9733333333333334 ± 0.024944382578492935 for model 2
Accuracy with priors [0.05, 0.05, 0.9]: 0.9666666666666668 ± 0.036514837167011066 for model 3


Aquí se ve que el resultado del modelo 2 anterior no era representativo. La precisión es ligeramente menor que en los modelos anteriores, lo que podría indicar que las muestras de la segunda clase no son tan fácilmente separables como las de la primera clase.

En cuanto al modelo 3: presenta la precisión más baja y la desviación estándar más alta, lo que podría sugerir que la tercera clase es la más difícil de clasificar correctamente o que el sesgo hacia esa clase introduce más incertidumbre en las predicciones.

Con respecto al modelo 0 y 1: cabe la posibilidad de pensar que por ejemplo, las características que definen la primera clase son muy distintas, el modelo podría seguir clasificando correctamente la mayoría de las instancias de esa clase, incluso si los priors cambian. Ello querría decir que dicha clase es fácilmente separable de las demás. Para obtener una visión más clara de qué sucediendo entre el modelo 1 y el 0, se puede ver la matriz de confusión.

In [5]:
from sklearn.metrics import confusion_matrix

def get_cm(model, test_x, test_y):
    test_preds_0 = model.predict(test_x)
    return confusion_matrix(test_y.T, test_preds_0.T)


X_full, y_full = get_iris_dataset()
a_priori_A = [1/3, 1/3, 1/3] # modelo 0
a_priori_B_1 = [0.9, 0.05, 0.05] # modelo 1
a_priori_B_2 = [0.05, 0.9, 0.05] # modelo 2
a_priori_B_3 = [0.05, 0.05, 0.9] # modelo 3

train_x, train_y, test_x, test_y = split_transpose(X_full, y_full, 0.4, 6543)

model = QDA()

model.fit(train_x, train_y, a_priori_A)
print("Matriz de confusión de prueba para el modelo 0:")
print(get_cm(model, test_x, test_y))

model.fit(train_x, train_y, a_priori_B_1)
print("Matriz de confusión de prueba para el modelo 1:")
print(get_cm(model, test_x, test_y))

Matriz de confusión de prueba para el modelo 0:
[[23  0  0]
 [ 0 20  1]
 [ 0  0 16]]
Matriz de confusión de prueba para el modelo 1:
[[23  0  0]
 [ 0 20  1]
 [ 0  0 16]]


El hecho de que el modelo 1, con priors sesgados ([0.9, 0.05, 0.05]), produzca una matriz de confusión similar indica que la información proporcionada por los datos (en este caso de la clase 1) es lo suficientemente fuerte como para compensar el sesgo de los priors.


## 2) Repetir punto 1 para el dataset penguin

In [6]:
from datasets import get_penguins
X_full_penguin, y_full_penguin = get_penguins()

print_distribution(y_full_penguin)

Distribución de clases:
Adelie: 0.4415
Chinstrap: 0.1988
Gentoo: 0.3596


Se puede observar este dataset no está balanceado con respecto a la cantidad de datos por clase.

In [7]:
priori_test(get_penguins())

A prioris:
 Adelie:0.333 , Chinstrap:0.333 , Gentoo:0.333 
[Model 0] Train (apparent) error is 0.0098 while test error is 0.0073
A prioris:
 Adelie:0.900 , Chinstrap:0.050 , Gentoo:0.050 
[Model 1] Train (apparent) error is 0.0195 while test error is 0.0219
A prioris:
 Adelie:0.050 , Chinstrap:0.900 , Gentoo:0.050 
[Model 2] Train (apparent) error is 0.0098 while test error is 0.0219
A prioris:
 Adelie:0.050 , Chinstrap:0.050 , Gentoo:0.900 
[Model 3] Train (apparent) error is 0.0098 while test error is 0.0073


Los modelos que mejor generalizan son el 0 y el 3. Parece ser un caso análogo al de Iris en cuanto a lo que sucede con diferentes priors. Se intentará verificar a continuación si esto es así:

In [8]:
priori_test_cross_val(get_penguins())

Accuracy with priors [0.3333333333333333, 0.3333333333333333, 0.3333333333333333]: 0.9882779198635976 ± 0.010993807100854342 for model 0
Accuracy with priors [0.9, 0.05, 0.05]: 0.9824381926683717 ± 0.005925745287640952 for model 1
Accuracy with priors [0.05, 0.9, 0.05]: 0.9589514066496164 ± 0.025287380244108995 for model 2
Accuracy with priors [0.05, 0.05, 0.9]: 0.9882779198635976 ± 0.010993807100854342 for model 3


Se puede ahora notar nuevamente que los modelos 1 (aunque por muy poco) y 2 son los que peor performan.

In [9]:
X_full, y_full = get_penguins()

train_x, train_y, test_x, test_y = split_transpose(X_full, y_full, 0.4, 6543)

model = QDA()

model.fit(train_x, train_y, a_priori_A)
print("Matriz de confusión de prueba para el modelo 0:")
print(get_cm(model, test_x, test_y))

model.fit(train_x, train_y, a_priori_B_1)
print("Matriz de confusión de prueba para el modelo 1:")
print(get_cm(model, test_x, test_y))

model.fit(train_x, train_y, a_priori_B_2)
print("Matriz de confusión de prueba para el modelo 2:")
print(get_cm(model, test_x, test_y))

model.fit(train_x, train_y, a_priori_B_3)
print("Matriz de confusión de prueba para el modelo 3:")
print(get_cm(model, test_x, test_y))

Matriz de confusión de prueba para el modelo 0:
[[67  0  0]
 [ 1 28  0]
 [ 0  0 41]]
Matriz de confusión de prueba para el modelo 1:
[[67  0  0]
 [ 3 26  0]
 [ 0  0 41]]
Matriz de confusión de prueba para el modelo 2:
[[64  3  0]
 [ 0 29  0]
 [ 0  0 41]]
Matriz de confusión de prueba para el modelo 3:
[[67  0  0]
 [ 1 28  0]
 [ 0  0 41]]


De aquí se puede concluir que existe una relación de compromiso entre clasificar correctamente las muestras de la primera y segunda clase. El límite de decisión que genera un error 0 requiera probablemente de mayor complejidad (trayendo aparejado un posible problema de overfitting) o siendo imposible de lograr.

El modelo 0 cuyos priors son los que más se asemejan a los reales, es uno de los que menor performa. El modelo 3 da resultados similares; ello se debe a que la tercera clase es evidentemente es fácilmente separable de las demás (puesto a que todos los modelos, a pesar de la abismal diferencia entre priors, han sido capaz de clasificarlos correctamente) y por lo tanto, la información proporcionada por los datos es lo suficientemente fuerte como para compensar el sesgo del prior (análogo al dataset anterior).

### 3) Implementar LDA

In [19]:
from utils import BaseBayesianClassifier, inv, det

class LDA(BaseBayesianClassifier):
    # Utiliza una matriz de covarianza ponderada para evitar que clases con mayor cantidad de muestras tengan mayor influencia en su valor final. 
    # Al ponderar, se intenta que la variabilidad dentro de la clase más pequeña también sea tenida en cuenta de manera proporcional.

  def _fit_params(self, X, y):
    # Inicializa la matriz de covarianza ponderada
    cov_matrix = np.zeros((X.shape[0], X.shape[0]))
    
    # Suma la covarianza de cada clase ponderada por su tamaño
    for idx in range(len(self.log_a_priori)):
        X_class = X[:, y.flatten() == idx]
        cov_matrix += np.cov(X_class, bias=True) * (X_class.shape[1] - 1)
    
    # Divide por el número total de muestras (menos 1 para corrección de Bessel)
    cov_matrix /= (X.shape[1] - len(self.log_a_priori))
    
    # Calcula la inversa de la matriz de covarianza ponderada
    self.inv_cov = inv(cov_matrix)
    self.means = [X[:,y.flatten()==idx].mean(axis=1, keepdims=True) for idx in range(len(self.log_a_priori))]


  def _predict_log_conditional(self, x, class_idx):
    # predict the log(P(x|G=class_idx)), the log of the conditional probability of x given the class
    # this should depend on the model used
    unbiased_x =  x - self.means[class_idx]
    return 0.5*np.log(det(self.inv_cov)) -0.5 * unbiased_x.T @ self.inv_cov @ unbiased_x

Se comparar LDA vs QDA (Sin multiples prioris, es decir que se estimen a partir de los datos) con los dos datasets:

In [23]:
for dataset_name, dataset in zip(['iris', 'penguins'], [get_iris_dataset(), get_penguins()]):
    for model_name, curr_model in zip(['QDA', 'LDA'], [QDA, LDA]):
        model = curr_model()
        x_full, y_full = dataset
        train_x, train_y, test_x, test_y = split_transpose(x_full, y_full, 0.4, 6543)
        model.fit(train_x, train_y)
        train_acc = accuracy(train_y, model.predict(train_x))
        test_acc = accuracy(test_y, model.predict(test_x))
        print(f"[Dataset={dataset_name}][Model={model_name}] train err {1-train_acc:.4f}, test err {1-test_acc:.4f}")

[Dataset=iris][Model=QDA] train err 0.0111, test err 0.0167
[Dataset=iris][Model=LDA] train err 0.0222, test err 0.0167
[Dataset=penguins][Model=QDA] train err 0.0146, test err 0.0146
[Dataset=penguins][Model=LDA] train err 0.0098, test err 0.0146


## FALTA CONCLUSIÓN!!!

Aclarar que QDA en general necesita más datos que LDA y que por eso quizás acá, no tenga mucho impacto el considerar cada matriz de covarianza por clase.

### 4) Utilizar otros 2 (dos) valores de *random seed* para obtener distintos splits de train y test, y repetir la comparación del punto anterior ¿Las conclusiones previas se mantienen?

In [24]:
import pandas as pd
df = pd.DataFrame()

for dataset_name,dataset in zip(['iris', 'penguins'],[get_iris_dataset(), get_penguins()]):
    for model_name, curr_model in zip(['QDA', 'LDA'],[QDA, LDA]):
        for seed in [6543, 5501,125]:
            model = curr_model()
            x_full, y_full = dataset
            train_x, train_y, test_x, test_y = split_transpose(x_full, y_full,test_sz=0.4, random_state=seed)
            model.fit(train_x, train_y)
            train_acc = accuracy(train_y, model.predict(train_x))
            test_acc = accuracy(test_y, model.predict(test_x))
            # print(f"[Dataset={dataset_name}][Model={model_name}] train err {1-train_acc:.4f}, test err {1-test_acc:.4f}")
            row = {
                'Dataset': dataset_name,
                'Model': model_name,
                'seed': seed,
                'Error (train)': 1-train_acc,
                'Error (test)': 1-test_acc,
            }
            
            df = pd.concat([df, pd.DataFrame([row])], ignore_index=True)
print(df)
    

     Dataset Model  seed  Error (train)  Error (test)
0       iris   QDA  6543       0.011111      0.016667
1       iris   QDA  5501       0.022222      0.016667
2       iris   QDA   125       0.022222      0.016667
3       iris   LDA  6543       0.022222      0.016667
4       iris   LDA  5501       0.022222      0.016667
5       iris   LDA   125       0.011111      0.016667
6   penguins   QDA  6543       0.014634      0.014599
7   penguins   QDA  5501       0.014634      0.007299
8   penguins   QDA   125       0.009756      0.014599
9   penguins   LDA  6543       0.009756      0.014599
10  penguins   LDA  5501       0.009756      0.014599
11  penguins   LDA   125       0.014634      0.007299


## FALTA CONCLUSIÓN!!!

### 5) Tensorized QDA vs QDA

In [25]:
from utils import TensorizedQDA

x_full, y_full = get_iris_dataset()
train_x, train_y, test_x, test_y = split_transpose(x_full, y_full,test_sz=0.4, random_state=6543)

tqda = TensorizedQDA()
tqda.fit(train_x, train_y)
train_acc = accuracy(train_y, tqda.predict(train_x))
test_acc = accuracy(test_y, tqda.predict(test_x))

print(f"Train (apparent) error is {1-train_acc:.4f} while test error is {1-test_acc:.4f}")

Train (apparent) error is 0.0111 while test error is 0.0167


In [26]:
%%timeit

tqda.predict(test_x)

860 μs ± 6.29 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [15]:
qda = QDA()
qda.fit(train_x, train_y)
train_acc = accuracy(train_y, qda.predict(train_x))
test_acc = accuracy(test_y, qda.predict(test_x))
print(f"Train (apparent) error is {1-train_acc:.4f} while test error is {1-test_acc:.4f}")

Train (apparent) error is 0.0111 while test error is 0.0167


In [16]:
%%timeit

qda.predict(test_x)

2.48 ms ± 49.7 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Tensorized QDA aprovecha que varias de las operaciones que se tienen que realizar se pueden escribir de forma matricial, pudiendo realizarse de forma paralela. Esto se hace evidente en las siguientes líneas:

```
self.tensor_inv_cov = np.stack(self.inv_covs)
self.tensor_means = np.stack(self.means)
```

Lo cual evidencia que las cuentas serán hechas simultáneamente, de forma paralela, para todas las clases y luego se verifica en la firma de la función:
```
def _predict_log_conditional(self, x, class_idx)
```

Ya que no es necesario especificar para qué clase será hecha dicha predicción.

## Faster QDA

In [17]:
from utils import TensorizedQDA
class FasterQDA(TensorizedQDA):
    
    def augment_dim(self, vec):
        return np.array([np.repeat(vec[i], 90) for i in range(len(self.log_a_priori))])
        
    
    def _predict_log_conditionals(self, x):
        # train_x era de 4x90
        # despues de hacer fit -> x es un tensor de 3 x 4 x 90 (o sea mismo array, 3 veces)
        # (hacemos la prediccion para las 3 clases en conjunto)
        unbiased_x = x - self.tensor_means
        # unbiased_x.shape = k x p x n 
        print(unbiased_x.shape)
        inner_prod = unbiased_x.transpose(0, 2, 1) @ self.tensor_inv_cov @ unbiased_x
        # inner prod shape: (k, n, n) -> k matrices de nxn
        # debemos de pensar que como es un producto de tensores => 
        # cada producto produce una matriz de nxn
        
        # dado un k fijo (estando parados en una clase)
        # 
        
        # podemos imaginar que en cada matriz
        
        # obtengo diagonal para cada matriz,
        # ya que solo las que son consigo mismas son las que nos interesan (producto interno)
        new_mat = np.array([np.diag(mat) for mat in inner_prod]) # (k,n)
        itcov = np.log(det(self.tensor_inv_cov)) # (k,)
        
        # truco para no tener que crear un nuevo vector itcov de (k,n) para poder sumarlo
        return 0.5 * itcov - 0.5 * new_mat.transpose() # (n,k)
    def predict(self, X):
        print(X.shape)        
        log_cond = self._predict_log_conditionals(x) # (n,k)
        log_priors = self.log_a_priori #(k,)
        y_hat = self.encoder.names[np.argmax(log_priors + log_cond, axis=1)]
        
        return y_hat
    
fqda = FasterQDA()
fqda.fit(train_x, train_y)
fqda.predict(train_x)

(4, 90)


NameError: name 'x' is not defined

In [ ]:
n = 90 # obs
p = 4 # features
k = 3 # classes
i_cov = np.ones((k,p,p)) # no se puede tocar
x = np.ones((k,p,n))
first = x.transpose(0,2,1)@i_cov
print(f'{(x.transpose(0,2,1)).shape} x {i_cov.shape} = {first.shape}')
print(f'{first.shape} x {x.shape} = {(first@x).shape} ')